# Churchland Widefield Data

This notebook demonstrates:
- Decoding L/R choice from different periods of the delay period

In [1]:
#Import Statements
import numpy as np
import pandas as pd
import ssm
import re
import seaborn as sns
import numpy.random as npr
import matplotlib.pyplot as plt
import numpy.linalg as la
from collections import OrderedDict
from matplotlib.font_manager import FontProperties
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from ssm import LDS
from Session import *
from ChoicePredictor import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
from matplotlib.cm import get_cmap
cmap = get_cmap('coolwarm')

# Single Session

## 0. Specify the mouse and session date you are looking at

In [3]:
mouse = "mSM36"
day = "05-Dec-2017"

## 1A. Load relevant classes

In [4]:
session = Session("vistrained", mouse, day)
predictor = SVCChoice(session, "behavenet")

## 1B. Fit polynomial-kernel SVC over each time step

In [ ]:
results = predictor.fit_all()

In [ ]:
# Save the logistic regression models
pickle.dump(results, open(
    "pickles/choicedecoding_behavelatents_" + mouse + "_" + day + ".p", "wb"
    ))

## 2. Plot decoding accuracy over time

In [ ]:
# Plot decoding accuracy over time
start_frame = []
accuracy = []
neur_accuracy = []
plt.figure(figsize=(9,7))
for frame, sc in enumerate(scores):
    start_frame.append(frame*2)
    accuracy.append(
        np.mean(sc)
        )
    
    # Now do the same for the full neural data
    neur_log_reg = neur_results[frame]
    neur_accuracy.append(
        np.max(np.mean(neur_log_reg.scores_[1], axis=0))
        )
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
plt.axvspan(30, 30+18, alpha=0.3, color='red')
plt.axvspan(30+33, 30+51, alpha=0.3, color='red')
plt.title("Choice Decoding")
plt.ylim((0.5,1))
sns.lineplot(
    x=start_frame, y=accuracy, label="BehaveNet Latents",
    linewidth=1.5
    )
sns.lineplot(
    x=start_frame, y=neur_accuracy, label="Original Neural Activity",
    linewidth=1.5
    )

## 3. Run LOO Decoding

In [ ]:
loo_results = predictor.fit_loo()

# All Sessions

## 1. Fit LR models for all sessions

In [ ]:
window_lengths = [2,4,6]
task = "vistrained"
taskdir = "/home/chingf/engram/data/musall/" + task + "/"
analysisdir = "/home/chingf/engram/analysis/behavenet/musall/" + task + "/"

def train_models(task, animal, day):
    """
    Fits and returns logistic regression models.
    
    Args:
        task: "vistrained" or "audtrained"
        animal: Name of the animal
        day: date of the session
    """
    
    # Load session and data
    session = Session(task, animal, day, access_engram=True)
    behave_latents = session.behavenet_latents
    trial_choices = session.trialmarkers['ResponseSide']
    delay_latents = []
    min_delay_size = 90
    delay_period_indices = session.get_delay_period(include_stim=True)
    for trial in range(delay_period_indices.shape[0]):
        start = delay_period_indices[trial,:][0] - 30
        end = delay_period_indices[trial,:][0] + 90
        activity = behave_latents[trial, start:end, :]
        delay_latents.append(activity)
    behave_latents = np.array(delay_latents)
    
    # Remove gap trials (just nans)
    behave_latents_nogap = []
    trial_choices_nogap = []
    for trial in range(session.num_trials):
        activity = behave_latents[trial,:,:]
        if np.sum(np.isnan(activity)) > 0:
            continue
        behave_latents_nogap.append(activity)
        trial_choices_nogap.append(trial_choices[trial])
    behave_latents = np.array(behave_latents_nogap)
    trial_choices = np.array(trial_choices_nogap)
    
    # Do a grid search over start index and window length
    # Fit logistic regression models
    all_results = []
    for window_length in window_lengths:
        start_idxs = range(0, behave_latents.shape[1], 2)
        window_results = [None for _ in range(session.num_bins)]
        for start_idx in start_idxs:
            log_reg = fit_LR(start_idx, window_length, behave_latents, trial_choices)
            window_results[start_idx] = log_reg
        all_results.append(window_results)
        
    # Collect the cross-validated accuracy
    accuracy_grid = []
    for idx, window_length in enumerate(window_lengths):
        window_results = all_results[idx]
        window_accuracy = []
        for results in window_results:
            if results == None:
                window_accuracy.append(0)#np.isnan)
                continue
            score = np.max(np.mean(results.scores_[1], axis=0))
            window_accuracy.append(score)
        accuracy_grid.append(window_accuracy)
    accuracy_grid = np.array(accuracy_grid)
    
    return all_results, accuracy_grid

In [ ]:
def fit_LR(start_idx, window_length, data, trial_choices):
    """
    Fits a L2-regularized logistic regression model, predicting
    left/right licking choice.
    
    Args
        start_idx: index in delay period to start extracting a window
            of activity.
        window_length: size of the window of activity to extract
    """
    
    X = []
    y = []
    # Extracting training and test data
    for trial in range(trial_choices.size):
        choice = trial_choices[trial]
        if np.isnan(choice):
            continue
        activity = data[trial, start_idx:start_idx+window_length, :]
        X.append(activity.flatten())
        y.append(int(choice-1))
    X = np.array(X)
    y = np.array(y)
    
    # Training the model with cross validation
    log_reg = LogisticRegressionCV(
        Cs=5, cv=5, scoring='accuracy',
        max_iter=500
        )
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    log_reg.fit(X, y)
    return log_reg

In [ ]:
# Run logistic regression over all available sessions
all_results = {}
all_accuracy = []
for animal in os.listdir(taskdir):
    if not animal.startswith("mSM"):
        continue
    if not animal in os.listdir(analysisdir):
        continue
    animaldir = taskdir + animal + "/"
    all_results[animal] = {}
    for day in os.listdir(animaldir):
        if not re.match("\d{2}-\w+-\d{4}", day):
            print("Invalid directory: " + day)
            continue
        if not day in os.listdir(analysisdir + animal + "/"):
            continue
        daydir = animaldir + day + "/"
        session_results, session_accuracy = train_models(task, animal, day)
        all_results[animal][day] = session_results

In [ ]:
# Save the logistic regression models
pickle.dump(all_results, open("temporaldecoding_behavenet_all.p", "wb" ) )

## 2. Decoding: Which time periods are most predictive?

In [ ]:
# If already available, load the logistic regression models
with open("temporaldecoding_behavenet_all.p", "rb") as f:
    all_results = pickle.load(f)

In [ ]:
# Collect and plot the 'accuracy grid' of each session
all_accuracy = []
for k1 in all_results.keys():
    for k2 in all_results[k1].keys():
        results = all_results[k1][k2]
        # Collect the cross-validated accuracy
        accuracy_grid = []
        for idx, window_length in enumerate(window_lengths):
            window_results = results[idx]
            window_accuracy = []
            for r in window_results:
                if r == None:
                    window_accuracy.append(0)#np.isnan)
                    continue
                score = np.max(np.mean(r.scores_[1], axis=0))
                window_accuracy.append(score)
            accuracy_grid.append(window_accuracy)
        accuracy_grid = np.array(accuracy_grid)
        all_accuracy.append(accuracy_grid)
        
#         # Plot the cross-validated accuracy
#         plt.figure(figsize=(10,8))
#         plt.imshow(accuracy_grid, cmap=cmap, aspect='auto', clim=(0.5, 1))
#         plt.colorbar()
#         plt.xlabel("Start Frame")
#         plt.ylabel("Framesize of Window")
#         plt.yticks(
#             ticks=np.arange(len(window_lengths)),
#             labels=[1*w for w in window_lengths]
#             )
#         plt.xticks(
#             ticks=np.arange(1, accuracy_grid.shape[1], 10),
#             labels=np.arange(1, accuracy_grid.shape[1], 10)
#             )
#         plt.title("Logistic Regression Accuracy\n" + k1 + "    " + k2)
#         plt.show()

        # Visualize accuracy just for window lengths of 2
        plt.figure()
        accuracy = []
        start_frame = []
        for idx, val in enumerate(accuracy_grid[0,:]):
            if val == 0:
                continue
            accuracy.append(val)
            start_frame.append(idx)
        plt.plot(start_frame, accuracy)
        plt.title("Decoding with window length of 2 frames\n" + k1 + "    " + k2)
        plt.ylabel("Choice Decoding Accuracy")
        plt.xlabel("Start Frame")
        plt.show()

In [ ]:
# Get the mean accuracy grid
mean_accuracy = np.zeros((len(window_lengths), 120))
accuracy_counts = np.zeros(mean_accuracy.shape)
for accuracy in all_accuracy:
    for i in range(accuracy.shape[0]):
        for j in range(120):
            mean_accuracy[i][j] += accuracy[i][j]
            accuracy_counts[i][j] += 1

mean_accuracy = np.divide(mean_accuracy, accuracy_counts)

In [ ]:
# Plot the cross-validated accuracy
plt.figure(figsize=(10,8))
plt.imshow(mean_accuracy, cmap=cmap, aspect='auto', clim=(0.5, 1))
plt.colorbar()
plt.xlabel("Start Frame")
plt.ylabel("Framesize of Window")
plt.yticks(
    ticks=np.arange(len(window_lengths)),
    labels=[1*w for w in window_lengths]
    )
plt.title("Logistic Regression Accuracy\nAveraged over all Sessions\n")
plt.show()

## 3. Decoding: Which regions are most predictive over time?

In [ ]:
# Visualize accuracy just for window lengths of 2

# Get all the window length 2 accuracies
accuracy_vals = []
start_frames = []
for accuracy in all_accuracy:
    for i in range(accuracy.shape[0]):
        for j in range(120):
            if accuracy[i][j] == 0:
                continue
            accuracy_vals.append(accuracy[i][j])
            start_frames.append(j)
df = pd.DataFrame({
    'Accuracy': accuracy_vals,
    'Frame': start_frames
    })

plt.figure(figsize=(9,7))
ax = sns.lineplot(x="Frame", y="Accuracy", data=df)
plt.title("Decoding with window length of 2 frames\n(All Sessions)", fontsize=14)
plt.ylabel("Choice Decoding Accuracy", fontsize=14)
plt.xlabel("Start Frame")
plt.xticks(
    ticks=[30, 30+18, 30+33, 30+51, 30+84],
    labels=['Stim On', 'Stim Off', 'Stim On', 'Stim Off', 'Spouts In'],
    rotation=45, fontsize=14
    )
ax.axvspan(30, 30+18, alpha=0.3, color='red')
ax.axvspan(30+33, 30+51, alpha=0.3, color='red')
plt.ylim((0.5,1))
plt.show()